In [7]:
import os
# Local libraries 
from data_analysis_utils.data_preprocessing import DataProcessor
from data_analysis_utils.nn_model_processing import NNModelProcessor

In [ ]:
'''
Data Processing
'''

# data_processor = DataProcessor(os.path.join("microclimate_data", "royal_city_daily_avg.csv")) # Daily averages
# data_processor = DataProcessor(os.path.join("microclimate_data", "royal_city_late_morning_avg.csv")) # Late morning averages (9:00AM - 11:00PM)
data_processor = DataProcessor(os.path.join("microclimate_data", "royal_city_all.csv"))    # 15-min data

# print(data_processor.get_data())
# data_processor.perform_eda()
# data_processor.pair_plots()

# Feature engineering (if necessary): Create new features or transform existing ones
# Data cleaning and preprocessing: Handle missing values, outliers, and inconsistencies

# Define the feature and target columns
feature_columns = ['time', 'wind_speed', 'solar_radiation', 'relative_humidity', 'air_temp', 'canopy_temp_mean']
target_columns = ['water_content_mean', 'water_potential_mean']

# Split and normalize the data
X_train_water_content, X_test_water_content, y_train_water_content, y_test_water_content = data_processor.split_and_normalize_data(feature_columns, 
                                                                                                                                   target_columns[0])

X_train_water_potential, X_test_water_potential, y_train_water_potential, y_test_water_potential = data_processor.split_and_normalize_data(feature_columns, 
                                                                                                                                           target_columns[1])

In [ ]:
'''
Defines and train neural network models for predicting water content and potential.
'''

model_processor = NNModelProcessor()

# Model for water content
model_nn_water_content = model_processor.define_tensorflow_keras_model_water_content(X_train_water_content.shape[1], 50, 0.0001)
results_water_content = model_processor.train_and_evaluate_tensorflow_keras_model(model_nn_water_content, 
                                                                                  X_train_water_content, 
                                                                                  y_train_water_content, 
                                                                                  X_test_water_content, 
                                                                                  y_test_water_content)

# Model for water potential
model_nn_water_potential = model_processor.define_tensorflow_keras_model_water_potential(X_train_water_potential.shape[1], 50, 0.001)
results_water_potential = model_processor.train_and_evaluate_tensorflow_keras_model(model_nn_water_potential, 
                                                                                    X_train_water_potential, 
                                                                                    y_train_water_potential, 
                                                                                    X_test_water_potential, 
                                                                                    y_test_water_potential)

In [ ]:
# Print results
model_processor.print_evaluation_results(results_water_content, "Water Content")
model_processor.print_evaluation_results(results_water_potential, "Water Potential")

In [ ]:
# Visualize results for water content prediction (scatter plots of predicted vs. actual values)
model_processor.visualize_results(y_test_water_content, results_water_content, "Water Content Prediction")
model_processor.visualize_results(y_test_water_potential, results_water_potential, "Water Potential Prediction")

In [ ]:
"""
Save, load, and evaluate trained models
"""

# Save trained models 
trained_models = {"water_content": model_nn_water_content, "water_potential": model_nn_water_potential}
model_processor.save_tensorflow_keras_models(models=trained_models)

# Load the models
loaded_models = model_processor.load_tensorflow_keras_models(models=trained_models)

# Evaluate models
evaluation_results_water_content = model_processor.evaluate_tensorflow_keras_model(loaded_models["water_content"], 
                                                                                   X_test_water_content, 
                                                                                   y_test_water_content)

evaluation_results_water_potential = model_processor.evaluate_tensorflow_keras_model(loaded_models["water_potential"], 
                                                                                     X_test_water_potential, 
                                                                                     y_test_water_potential)

# Print evaluation results
model_processor.print_evaluation_results(evaluation_results_water_content, "Water Content")
model_processor.print_evaluation_results(evaluation_results_water_potential, "Water Potential")